# ReporterScreen sample / guide quality report

Examine the quality of the guide and samples and masks the low-quality guides and samples.

In [ ]:
import perturb_tools as pt
import bean as be
from bean.qc.utils import fill_in_missing_samples
import matplotlib.pyplot as plt
plt.style.use('default')

In [ ]:
exp_id = "LDLRCDS"
save_format = "png"
bdata_path = "../results/mapped/LDLRCDS/bean_count_LDLRCDS_combined.h5ad"
out_bdata_path = "../results/mapped/LDLRCDS/bean_count_LDLRCDS_masked.h5ad"
rel_pos_is_reporter=False
edit_quantification_start_pos = 2
edit_quantification_end_pos = 7
target_pos_col = "target_pos"
corr_X_thres = 0.7
edit_rate_thres = 0.1
posctrl_col = "group"
posctrl_val = "PosCtrl"
lfc_thres = -0.1
replicate_label = "rep"
condition_label = "bin"
comp_cond1 = "top"
comp_cond2 = "bot"
recalculate_edits = False
tiling = None

In [ ]:
bdata = be.read_h5ad(bdata_path)

In [ ]:
if tiling is not None:
    bdata.uns['tiling'] = tiling

Add dummy samples if not paired

In [ ]:
bdata = fill_in_missing_samples(bdata, condition_label, replicate_label)

In [ ]:
bdata.samples

In [ ]:
bdata.X

Annotate unannotated samples & log-normalize guides

In [ ]:
#bdata.samples[[replicate_label, condition_label]] = bdata.samples.index.to_series().str.split("_", expand=True)

In [ ]:
bdata.log_norm()

## Sample quality

### Visualize quality metrics

#### 1. Guide coverage

In [ ]:
pt.qc.plot_guide_coverage(bdata, figsize=(6,4))


In [ ]:
plt.style.use('default')
pt.qc.plot_X_gini(bdata)
plt.savefig(f"{exp_id}_gini.{save_format}")

#### 2. Guide abundance correlation

In [ ]:
pt.qc.plot_correlation(bdata, "Spearman")

#### 3. LFC correlation of positive controls

In [ ]:
selected_guides = bdata.guides[posctrl_col] == posctrl_val if posctrl_col else ~bdata.guides.index.isnull()
ax=pt.qc.plot_lfc_correlation(bdata, selected_guides, method="Spearman", cond1=comp_cond1, cond2=comp_cond2, rep_col=replicate_label, compare_col=condition_label, figsize=(10,10))

ax.set_title("top/bot LFC correlation, Spearman")
plt.yticks(rotation=0) 
plt.xticks(rotation=90) 
plt.show()

#### 4. Guide editing rates

In [ ]:
if recalculate_edits or "edits" not in bdata.layers.keys() or bdata.layers['edits'].max() == 0:
    if 'allele_counts' in bdata.uns.keys():
        bdata.uns['allele_counts'] = bdata.uns['allele_counts'].loc[bdata.uns['allele_counts'].allele.map(str) != ""]
        bdata.get_edit_from_allele()
        bdata.get_edit_mat_from_uns(
            rel_pos_start=edit_quantification_start_pos, 
            target_pos_col=target_pos_col,
            rel_pos_end=edit_quantification_end_pos, 
            rel_pos_is_reporter=rel_pos_is_reporter
        )

In [ ]:
if "edits" in bdata.layers.keys():
    bdata.get_guide_edit_rate(
        editable_base_start = edit_quantification_start_pos, 
        editable_base_end=edit_quantification_end_pos)
    be.qc.plot_guide_edit_rates(bdata)

In [ ]:
if "edits" in bdata.layers.keys():
    bdata.get_edit_rate(
        editable_base_start = edit_quantification_start_pos, 
        editable_base_end=edit_quantification_end_pos
    )
    be.qc.plot_sample_edit_rates(bdata)

### Mask low-quality samples

In [ ]:
bdata.samples.style.background_gradient(cmap="coolwarm_r")

Assign sample mask to mask low-quality samples.

In [ ]:
bdata.samples['mask'] = 1
bdata.samples.loc[bdata.samples.median_corr_X < corr_X_thres, 'mask'] = 0
if "median_editing_rate" in bdata.samples.columns.tolist():
    bdata.samples.loc[bdata.samples.median_editing_rate < edit_rate_thres, 'mask'] = 0
bdata_filtered = bdata[:, bdata.samples["median_lfc_corr.top_bot"] > lfc_thres]

In [ ]:
# leave replicate with more than 1 sorting bin data
rep_n_samples = bdata_filtered.samples.groupby(replicate_label)['mask'].sum()
print(rep_n_samples)
rep_has_too_small_sample = rep_n_samples.loc[rep_n_samples < 2].index.tolist()
rep_has_too_small_sample
print(f"Excluding reps {rep_has_too_small_sample} that has less than 2 samples per replicate.")
bdata_filtered = bdata_filtered[:, ~bdata_filtered.samples[replicate_label].isin(rep_has_too_small_sample)]

In [ ]:
bdata_filtered.samples.style.background_gradient(cmap="coolwarm_r")

## Identify outlier guides

In [ ]:
outlier_guides, mask = be.qc.get_outlier_guides_and_mask(bdata_filtered, condit_col = condition_label, replicate_col = replicate_label)

In [ ]:
outlier_guides


In [ ]:
outlier_guides_n_samples = outlier_guides['name'].value_counts()
guides_to_exclude = outlier_guides_n_samples.loc[outlier_guides_n_samples > 2].index
guides_to_exclude

In [ ]:
bdata_filtered.uns['repguide_mask'] = mask

In [ ]:
bdata_filtered = bdata_filtered[~bdata_filtered.guides.index.isin(guides_to_exclude),:]

In [ ]:
bdata_filtered

In [ ]:
bdata_filtered.uns['repguide_mask'].shape

In [ ]:
bdata_filtered.write(out_bdata_path)